In [24]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from loaders import TrnLoader
from train_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
iterates_to_combine = list(range(58, 64)) #[7, 8, 9, 10]
model_stem = '1.29'
#run_id = 'kszjvkvy'

In [6]:
#glob.glob(f"{BESPOKE_ROOT}/models/m_{model_stem}*")

In [7]:
m_paths = [f"{BESPOKE_ROOT}/models/m{model_stem}_e{e}.torch" for e in iterates_to_combine]
m_paths

['/home/beans/bespoke/models/m1.29_e58.torch',
 '/home/beans/bespoke/models/m1.29_e59.torch',
 '/home/beans/bespoke/models/m1.29_e60.torch',
 '/home/beans/bespoke/models/m1.29_e61.torch',
 '/home/beans/bespoke/models/m1.29_e62.torch',
 '/home/beans/bespoke/models/m1.29_e63.torch']

In [8]:
state_dicts = [torch.load(p) for p in m_paths]
state_dict_avg = state_dicts[0].copy() # placeholder for a sec

In [9]:
for k in state_dict_avg.keys():
    state_dict_avg[k] = sum([sd[k] for sd in state_dicts]) / len(state_dicts)

In [10]:
m = EffNet().to(device)
m.train()
m.load_state_dict(state_dict_avg)

<All keys matched successfully>

In [15]:
N_UPDATES = 256
for mm in m.modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = 1/N_UPDATES

In [12]:
%%time
BS = 64 # 128 got an error?
dataloader = TrnLoader(bs=BS)

Launching 3 loader workers


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated.

Got first chunk
CPU times: user 70.7 ms, sys: 344 ms, total: 414 ms
Wall time: 2.52 s


Process Process-3:
Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 130, in make_chunks
    time.sleep(.1)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 1

In [14]:
trainer = Trainer(dataloader, m, model_stem=model_stem, opt=None, log_wandb=False, updates_per_epoch=N_UPDATES,
                  backwards=False, total_epochs=1)

In [16]:
%%time 
with torch.no_grad():
    trainer.train()

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")




 {'avg_unc': -7.47769165, 'wp_angles_i': 0.00213718, 'wp_headings_i': 0.00199401, 'wp_curvatures_i': 0.01404866, 'wp_rolls_i': 4.44e-05, 'wp_zs_i': 0.00053997, 'wp_angles': 0.00108046, 'wp_headings': 0.00144699, 'wp_curvatures': 0.01001991, 'wp_rolls': 0.07793486, 'wp_zs': 0.06245124, 'has_stop': 0.01225658, 'stop_dist': 0.00472811, 'has_lead': 0.00711477, 'lead_dist': 0.00189534, 'lead_speed': 0.26070365, 'dagger_shift': 0.03662863, 'lane_width': 0.05031168, 'rd_is_lined': 0.00065406, 'left_turn': 0.0025204, 'right_turn': 0.0005163, 'unc': 1.18574142, 'logistical/obs_consumed_per_second': 110.04724409, 'logistical/obs_generated_per_second': 75.94488189, 'logistical/slowest_runner_obs_per_sec': 1.75590551, 'logistical/data_consumption_ratio': 1.44788822, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.29817848, 'timing/model forward': 0.40682835, 'timing/calc losses': 0.00812063, 'timing/backwards': 6.19e-06, 'timing/get worst': 2.02e-06, 'timing/logging':

In [17]:
for mm in m.modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .1 # set this back to value used for train. Actually is this even necessary? is this stored in state_dict?
        #print(mm.momentum)

In [18]:
torch.save(m.state_dict(), f'{BESPOKE_ROOT}/models/m{model_stem}_avg.torch')

In [19]:
m.model_stem = f"{model_stem}_avg" # rw evaluator uses this

In [20]:
wandb.init(id='66i4vxlw', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [25]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(m, wandb=wandb, save_rollouts=True)

CPU times: user 9 µs, sys: 3 µs, total: 12 µs
Wall time: 13.1 µs


In [26]:
rw_evaluator.evaluate()

0


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning:

None of the inputs have requires_grad=True. Gradients will be None



3200
loader is done
Rollout complete!
0
3200
loader is done
Rollout complete!
0
3200
loader is done
Rollout complete!
0
loader is done
Rollout complete!
0
3200
loader is done
Rollout complete!
0
3200
loader is done
Rollout complete!
down w rollouts, reporting
run_555a
run_556a
run_556b
run_556c
run_555b
run_556d
